<a href="https://colab.research.google.com/github/KumarNavish/Cryptocurrency-Prediction-using-RNNs/blob/master/Feature_Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install contractions

import json
import pickle
import re
import contractions
import unicodedata
from tqdm import tqdm

import numpy as np
# import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')

In [0]:
######################## LOAD DATASET ######################

# The file is being fetched from remote - the following needs to run only once

import requests
response_dataset = requests.get(url="https://raw.githubusercontent.com/hackathoniima/ICADABAI2019/master/dataset.json")
data = response_dataset.json()

# Alterativly you can load the file from local as well by the following code.
# To do so, comment out the above and uncomment the below.
#
# with open("dataset.json", "r") as fil:
#   data = json.load(fil)

In [0]:
data[:2]

['Let me explain to you why I believe this : For years, SunTrust advertised and promoted in big letters their XXXX XXXX debit card.',
 'With that card, an account holder could earn XXXX mile for every {$1.00} spent ( for debit and credit card transactions ).']

In [0]:
regex_for_XXX = r'[^a-zA-Z0-9]{1}[X][X \/-]{0,}[X]'


In [0]:
########################## Check for the pattern XXXX ##########################

regex_for_XXX = r'[^a-zA-Z0-9]{1}[X][X \/-]{0,}[X]'
m = []
new_data = []
find_start_index = []
match_XXXX_len = []
for row in data: 
  match = re.findall(regex_for_XXX,row)
#   print(match)
  if len(match) != 1:
    match2 = re.findall("[X][X \/-]{0,}[X]",row)
    if len(match2) != 1:
      print(row)
      continue
    else:
      match2 = re.findall("[X][X \/-]{0,}[X]",match2[0])

      if match2[0] not in m:
        m.append(match2[0])

      new_data.append(row)
      find_start_index.append(re.search(match2[0],row).start())
      match_XXXX_len.append(len(match2[0]))

  else:
    match2 = re.findall("[X][X \/-]{0,}[X]",match[0])
    find_start_index.append(re.search(match2[0],row).start())
    match_XXXX_len.append(len(match2[0]))

    if match2[0] not in m:
      m.append(match2[0])

    new_data.append(row)

data = new_data

"""
Preprocessing: BASIC WORKFLOW

#     Split about '-' or '_'
    sents = sents.replace('-',' - ').replace('_',' _ ')
#     I'm -> I am || gotta -> got to et al
    sents = contractions.fix(sents) #, slang = True)
#     tokenize
#     Number to words
    sents = replace_numbers(sents)
#     Remove non-ASCII Chars, emoji ...
    sents = remove_non_ascii(sents)
#     To lowercase ############# Commented out for now
    sents = to_towercase(sents)
#     Remove punctuation
    sents = rm_punc(sents)
#     Remove stopwords
    sents = rm_stopwords(sents)
#     Lemmatize
    sents = lemmatize_verbs(sents)
#     sents = sents.lower()
"""

In [0]:
def remove_non_ascii(words): ###### Remove non-ASCII characters from list of tokenized words
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

In [0]:
def to_towercase(words): ###### Convert all characters to lowercase from tokenized list
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [0]:
def rm_punc(sents): #rm punctuation from a list of words
    new_words = []
    for word in sents:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [0]:
def rm_stopwords(sents):
    stopwords_list = nltk.corpus.stopwords.words('english')
    new_words = []
    for word in sents:
        if word not in stopwords_list:
            new_words.append(word)
    return(new_words)

In [0]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [0]:
def preprocess_string(sents):
  
  
  # Split about '-' or '_'
  sents = sents.replace('-',' - ').replace('_',' _ ')

  # I'm -> I am || gotta -> got to 
  sents = contractions.fix(sents) #, slang = True)

  # Tokenize
  sents = nltk.word_tokenize(sents)
    
  # Remove non-ASCII Chars, emoji ...
  sents = remove_non_ascii(sents)

  # To lowercase
#   sents = to_towercase(sents)

  # Remove punctuation
  sents = rm_punc(sents)
  
  # Remove stopwords
#   sents = rm_stopwords(sents)

  # Lemmatize
  sents = lemmatize_verbs(sents)
  
  return sents

############################## End of preprocessing ###########################

########## TO NOTE #########
* Stopwords
* Lowercase

In [0]:
############## Divide each string of training data about the matched part for pattern -XXXX ##########

# formatted_data = []

# for i in range(len(data)):
#   row = data[i]
#   this_entity = {}
#   this_entity["left_part"] = preprocess_string((row[ : find_start_index[i]]).strip())
#   this_entity["matched_part"] = (row[find_start_index[i] : find_start_index[i] +
#                                                      match_XXXX_len [i]]).strip()
#   this_entity["right_part"] = preprocess_string((row[find_start_index[i] +
#                                         match_XXXX_len [i] : ]).strip())
#   formatted_data.append(this_entity)

# with open('formatted_data.pkl','wb') as fp:
#     pickle.dump(formatted_data,fp)

### Loading the Saved data | Run  from hereon after

In [0]:
with open('formatted_data.pkl','rb') as fp:
    formatted_data=pickle.load(fp)
formatted_data[0]

{'left_part': ['Let',
  'me',
  'explain',
  'to',
  'you',
  'why',
  'I',
  'believe',
  'this',
  'For',
  'years',
  'SunTrust',
  'advertise',
  'and',
  'promote',
  'in',
  'big',
  'letter',
  'their'],
 'matched_part': 'XXXX XXXX',
 'right_part': ['debit', 'card']}

* !wget http://nlp.stanford.edu/data/glove.6B.zip
* !unzip glove*.zip

## Word2Vec_embedding Dict:

In [0]:
# print('Indexing word vectors.')

# embeddings_index = {}
# with open('glove.6B.50d.txt', encoding='utf-8') as fp:
#     for line in fp:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:],dtype=np.float32)
#         embeddings_index[word] = coefs

# with open('embeddings_index.pkl','wb') as fp:
#     pickle.dump(embeddings_index,fp)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
with open('embeddings_index.pkl','rb') as fp:
    embeddings_index=pickle.load(fp)
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


## Applying padding in sequence lengths (required for dynamic_rnn_input)

In [0]:
#Finding Max sequence length in both left & right context vector
l,r = zip(*[(len(formatted_data[i]['left_part']),len(formatted_data[i]['right_part'])) for i in range(len(formatted_data))])
max_l,max_r = max(l),max(r)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
vector_len=50
# Converting Word to Vectors (if words are present in pretrained model):
def create_word2vec(data):
#     context_vec=[]
    with open('context_vec.pkl','wb') as fp:
        for index,i in tqdm(enumerate(data)):
            left=right=[]

            # For Left Part
            left = [list(embeddings_index[j]) for j in i['left_part'] if j in embeddings_index.keys()]
            # Padding | making same length sequences
            if len(left)<max_l:
                left += np.zeros((max_l-len(left),vector_len),dtype=np.float32).tolist()


            # For right part   
            right = [list(embeddings_index[j]) for j in i['right_part'] if j in embeddings_index.keys()]
            # Padding | making same length sequences
            if len(right)<max_r:
                right += np.zeros((max_r-len(right),vector_len),dtype=np.float32).tolist() 
                
            pickle.dump({'left_ContextVec':left,'right_ContextVec':right},fp)            
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
create_word2vec(formatted_data)

30916it [01:28, 348.80it/s]


In [0]:
context_vec = []# is huge in memory ~ 25.6gb
i=0
with open('context_vec.pkl', 'rb') as fr:
    try:
        while i<4:
            context_vec.append(pickle.load(fr))
            i+=1
    except EOFError:
        pass

In [0]:
len(context_vec)

4

## After Preprocessing and word2vec generation- Further Process

In [0]:
import tensorflow as tf

* This is the recommended initializer for NN weights:
* tf.truncated_normal_initializer()

## Left and Right Context Vectors(*not complete):

In [0]:
left_cv = np.array([np.array(context_vec[i]['left_ContextVec'],dtype=np.float32) for i in range(4)])
right_cv = np.array([np.array(context_vec[i]['right_ContextVec'],dtype=np.float32) for i in range(4)])

## Bidirectional LSTM, Dynamic RNN

In [0]:
rnn_size, keep_prob = 64,0.85
def bidirectional_lstm(input_data, rnn_size, keep_prob):
#     with open('context_vec.pkl', 'rb') as fr:
#         try:
#             while True:
#                 output.append(input_data)
#         except EOFError:
#             pass
            
    with tf.variable_scope('Hello',reuse=tf.AUTO_REUSE):

        # forward input embedding
        cell_fw = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.truncated_normal_initializer(-0.1, 0.1, seed=2))
        cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob = keep_prob)

        # backward input embedding
        cell_bw = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.truncated_normal_initializer(-0.1, 0.1, seed=2))
        cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob = keep_prob)

        # Outputs is a tuple of backward and forward input embedding
        outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                          cell_bw, 
                                                          input_data,
                                                          dtype=tf.float32)
    return outputs

### Building the Session

In [0]:
inputs = tf.placeholder(dtype=tf.float32, shape=left_cv.shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    left_hidden_fw,left_hidden_bw = sess.run(bidirectional_lstm(inputs, rnn_size, keep_prob), 
                                        {inputs:left_cv})
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
inputs = tf.placeholder(dtype=tf.float32, shape=right_cv.shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    right_hidden_fw,right_hidden_bw = sess.run(bidirectional_lstm(inputs, rnn_size, keep_prob), 
                                        {inputs:right_cv})

In [0]:
left_hidden_fw[0]

array([[-1.40927257e-02, -9.69089940e-02,  6.76873028e-02, ...,
        -5.02436385e-02, -4.39806134e-02,  1.00969449e-01],
       [ 7.03788316e-03, -1.11870885e-01, -8.80033821e-02, ...,
        -1.58782095e-01, -6.47772253e-02, -3.22080031e-02],
       [ 7.72010088e-02,  5.02228737e-03, -4.52372245e-02, ...,
        -1.75192028e-01,  9.27934796e-02, -1.32149622e-01],
       ...,
       [-9.20629617e-09,  1.43528177e-06, -2.07196854e-06, ...,
        -3.51621935e-07, -1.63170353e-06,  1.21182529e-06],
       [-1.23268808e-08,  1.30279216e-06, -1.87978276e-06, ...,
        -3.04943285e-07, -1.42877650e-06,  1.16133344e-06],
       [-5.99632699e-08,  1.14378372e-06, -1.73965395e-06, ...,
        -3.00651777e-07, -1.28322768e-06,  1.07255380e-06]], dtype=float32)

## Pairwise ordering of fw and bw hidden vectors

In [0]:
batch = left_hidden_fw.shape[0]
def input_format(batch, fw, bw):
    merged = []
    for i in range(left_hidden_fw.shape[0]):
        temp = np.zeros(((2*fw.shape[1]),fw.shape[2]),dtype=np.float32)
        for j in range(fw.shape[1]):
            temp[2*j]= fw[i][j]
            temp[2*j+1] = bw[i][j]
        merged.append(temp)
    return np.array(merged)
left_hidden_merged = input_format(batch, left_hidden_fw, left_hidden_bw)
right_hidden_merged = input_format(batch, right_hidden_fw, right_hidden_bw )

In [0]:
right_hidden_merged[0]

array([[-7.56302699e-02, -7.51577690e-02, -1.42197892e-01, ...,
        -2.02055603e-01,  1.58784941e-01,  1.28846183e-01],
       [-1.76053941e-01, -8.22217204e-03,  2.00145263e-02, ...,
        -3.24034572e-01,  2.26524055e-01, -6.02251925e-02],
       [-1.41308442e-01,  6.39979020e-02, -8.28666165e-02, ...,
        -2.94512570e-01,  3.33593398e-01,  7.43647665e-02],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.40627865e-08,  6.08729920e-08, -5.29557376e-08, ...,
        -3.29368688e-09, -5.54569617e-08,  4.76004587e-08],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

## This Code Block is under construction

In [0]:
dim_hidden_state,num_hidden_layer1 = 100, 64
weights = {'e_left': tf.Variable(tf.random_normal([dim_hidden_state,num_hidden_layer1])),
           'e_right': tf.Variable(tf.random_normal([dim_hidden_state,num_hidden_layer1])),
           'a_left': tf.Variable(tf.random_normal([num_hidden_layer1,1])),
           'a_right': tf.Variable(tf.random_normal([num_hidden_layer1,1]))}

# NN with 1 hidden Layer:
e_l = tf.nn.tanh(tf.matmul(new_outputs,weights['e_left']))
e_r = tf.nn.tanh(tf.matmul(new_outputs,weights['e_right']))

# Multiplying the above vectors(dim=(30916,num_hidden_layer1)) with W_a's to get a scalar
e_l = tf.nn.exp(tf.matmul(e_l,weights['a_left']))
e_r = tf.nn.exp(tf.matmul(e_r,weights['a_right']))

# Normalizing above scalars:
a_l = e_l/(e_l + e_r)
a_r = e_r/(e_l + e_r)

# Final context vector:
V_c = []
for i,j in zip(a_l,a_r):
    V_c.append(tf.add(tf.scalar_mul(i,left_context_vec),tf.scalar_mul(j,right_context_vec)))
V_c = tf.array(V_c)
